In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score,KFold
from sklearn.ensemble import GradientBoostingRegressor

import warnings


warnings.filterwarnings("ignore")

# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

In [5]:
X = train_df.loc[:,'Id':'SaleCondition_Partial']
y = train_df['SalePrice']
print("Shape of training set {}.\nShape of test set {}".format(X.shape,y.shape))

Shape of training set (1460, 303).
Shape of test set (1460,)


In [7]:
gBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.03,
                                   max_depth=3, max_features='sqrt', 
                                   loss='huber', random_state =5)

kf = KFold(10, shuffle=True, random_state=42).get_n_splits(X)
rmse_cv_gB= np.sqrt(-cross_val_score(gBoost, X, y, scoring="neg_mean_squared_error", cv = kf))
print("The 10-fold crossvalidation RMSE of GBoost is {:.5f} +/- {:.3f}".format(rmse_cv_gB.mean(),
                                                                               rmse_cv_gB.std()))

The 10-fold crossvalidation RMSE of GBoost is 0.11381 +/- 0.018


In [ ]:
#Prediction on real test set using KRR
KRR.fit(X,y)
pred_results =KRR.predict(test_df)
result_df = pd.DataFrame(data={'Id': test_df["Id"].values,
                               'SalePrice': pred_results})
#Create output csv file
result_df.to_csv(data_path+"outputs/kernel_ridge_a_1", index=False)